In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Rescaling, Flatten, Activation
from tensorflow.keras.models import Sequential


In [ ]:
image_size = (48,48)
batch_size = 32

trainingSet = tf.keras.preprocessing.image_dataset_from_directory(
    "fer_images/Training",  
    seed = 1337, 
    image_size = image_size, 
    batch_size = batch_size,
)

validationSet = tf.keras.preprocessing.image_dataset_from_directory(
    "fer_images/PrivateTest", 
    seed = 1337, 
    image_size = image_size, 
    batch_size = batch_size,
)

testSet = tf.keras.preprocessing.image_dataset_from_directory(
    "fer_images/PublicTest", 
    seed = 1337, 
    image_size = image_size, 
    batch_size = batch_size,
)



In [ ]:
# %%
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 12))
for images, labels in trainingSet.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")
plt.show()

In [ ]:
def my_model(input_shape, num_classes):
    model = Sequential()

    # Entry block
    model.add(Rescaling(1.0 / 255))
    model.add(Conv2D(64, (5, 5), strides=2, padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPool2D(pool_size=3, strides=2))


    model.add(Conv2D(64, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPool2D(pool_size=3, strides=2))

    model.add(Conv2D(128, (4,4), padding="same"))
    model.add(Activation("relu"))

    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(
        units = 3072, 
        activation = 'relu'
    ))

    model.add(Dense(7, activation='softmax'))
    epochs = 100

    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    model.fit(
        trainingSet, epochs=epochs, batch_size = 16, validation_data=validationSet,
    )
    
    return model

In [ ]:
model = my_model(image_size, 7)
model.save('model.h5')